In [1]:
import torch

if torch.cuda.is_available():
  device = torch.device('cuda')
  print('There are %d GPU(s) available' % torch.cuda.device_count())
  print(torch.cuda.get_device_name(0))
else:
  device = torch.device('cpu')
  print('No GPU available, CPU instaed')

device

No GPU available, CPU instaed


device(type='cpu')

In [2]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
import pandas as pd
import numpy as np
from copy import deepcopy
import re
from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split

import torch
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModel
from transformers import BertForSequenceClassification, AdamW, BertConfig

import numpy as np
import datetime
import time
import random
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix, \
                            precision_score, recall_score, accuracy_score, roc_auc_score, average_precision_score

In [3]:
lenta_df = pd.read_excel('lenta_finam.xlsx')
rbk_df = pd.read_excel('rbk_finam.xlsx')

In [4]:
lenta_df_ton = pd.read_csv('lenta_finam_unique_sentiment.xlsx')
rbk_df_ton = pd.read_csv('rbk_finam_unique_sentiment.xlsx')

In [5]:
lenta_df.head()

,Unnamed: 0,docid,url,title,modified,lastmodtime,type,domain,status,part,...,TICKER,PER,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,date_y
0,0,1181788,https://lenta.ru/news/2022/01/03/sroki/,В МИД России назвали сроки сертификации «Север...,1641201720,1641204328,1,1,0,0,...,SBER,60,20220103,10:00,"1,326.2","1,330.5","1,321.7","1,323.1","22,990",2022-01-03 10:00:00
1,1,1181788,https://lenta.ru/news/2022/01/03/sroki/,В МИД России назвали сроки сертификации «Север...,1641201720,1641204328,1,1,0,0,...,ROSN,60,20220103,10:00,"1,326.2","1,330.5","1,321.7","1,323.1","22,990",2022-01-03 10:00:00
2,2,1181788,https://lenta.ru/news/2022/01/03/sroki/,В МИД России назвали сроки сертификации «Север...,1641201720,1641204328,1,1,0,0,...,MOEX,60,20220103,10:00,"1,326.2","1,330.5","1,321.7","1,323.1","22,990",2022-01-03 10:00:00
3,3,1181788,https://lenta.ru/news/2022/01/03/sroki/,В МИД России назвали сроки сертификации «Север...,1641201720,1641204328,1,1,0,0,...,YNDX,60,20220103,10:00,"1,326.2","1,330.5","1,321.7","1,323.1","22,990",2022-01-03 10:00:00
4,4,1181788,https://lenta.ru/news/2022/01/03/sroki/,В МИД России назвали сроки сертификации «Север...,1641201720,1641204328,1,1,0,0,...,PIKK,60,20220103,10:00,"1,326.2","1,330.5","1,321.7","1,323.1","22,990",2022-01-03 10:00:00


In [6]:
lenta_df_ton.head()

,title,sentiment
0,В МИД России назвали сроки сертификации «Север...,отрицательный
1,Российской экономике предсказали замедление,отрицательный
2,Российские миллиардеры стали еще богаче,отрицательный
3,США увеличили импорт мороженого из России в не...,отрицательный
4,Производитель одежды для Nike и Adidas потерял...,отрицательный


In [7]:
lenta_df = lenta_df.merge(lenta_df_ton, left_on='title', right_on='title').drop(['Unnamed: 0'], axis=1)
rbk_df = rbk_df.merge(rbk_df_ton, left_on='title', right_on='title').drop(['Unnamed: 0'], axis=1)

In [8]:
lenta_df.head()

,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,...,PER,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,date_y,sentiment
0,1181788,https://lenta.ru/news/2022/01/03/sroki/,В МИД России назвали сроки сертификации «Север...,1641201720,1641204328,1,1,0,0,4,...,60,20220103,10:00,"1,326.2","1,330.5","1,321.7","1,323.1","22,990",2022-01-03 10:00:00,отрицательный
1,1181788,https://lenta.ru/news/2022/01/03/sroki/,В МИД России назвали сроки сертификации «Север...,1641201720,1641204328,1,1,0,0,4,...,60,20220103,10:00,"1,326.2","1,330.5","1,321.7","1,323.1","22,990",2022-01-03 10:00:00,отрицательный
2,1181788,https://lenta.ru/news/2022/01/03/sroki/,В МИД России назвали сроки сертификации «Север...,1641201720,1641204328,1,1,0,0,4,...,60,20220103,10:00,"1,326.2","1,330.5","1,321.7","1,323.1","22,990",2022-01-03 10:00:00,отрицательный
3,1181788,https://lenta.ru/news/2022/01/03/sroki/,В МИД России назвали сроки сертификации «Север...,1641201720,1641204328,1,1,0,0,4,...,60,20220103,10:00,"1,326.2","1,330.5","1,321.7","1,323.1","22,990",2022-01-03 10:00:00,отрицательный
4,1181788,https://lenta.ru/news/2022/01/03/sroki/,В МИД России назвали сроки сертификации «Север...,1641201720,1641204328,1,1,0,0,4,...,60,20220103,10:00,"1,326.2","1,330.5","1,321.7","1,323.1","22,990",2022-01-03 10:00:00,отрицательный


In [9]:
rbk_df[['id', 'category', 'title', 'body', 'text']]

,id,category,title,body,text
0,61d4b1139a79478571789993,Политика,Токаев ввел чрезвычайное положение в Мангистау...,Чрезвычайное положение вступит в силу 5 января...,Президент Казахстана Касым-Жомарт Токаев подпи...
1,61d4b1139a79478571789993,Политика,Токаев ввел чрезвычайное положение в Мангистау...,Чрезвычайное положение вступит в силу 5 января...,Президент Казахстана Касым-Жомарт Токаев подпи...
2,61d4b1139a79478571789993,Политика,Токаев ввел чрезвычайное положение в Мангистау...,Чрезвычайное положение вступит в силу 5 января...,Президент Казахстана Касым-Жомарт Токаев подпи...
3,61d4b1139a79478571789993,Политика,Токаев ввел чрезвычайное положение в Мангистау...,Чрезвычайное положение вступит в силу 5 января...,Президент Казахстана Касым-Жомарт Токаев подпи...
4,61d4b1139a79478571789993,Политика,Токаев ввел чрезвычайное положение в Мангистау...,Чрезвычайное положение вступит в силу 5 января...,Президент Казахстана Касым-Жомарт Токаев подпи...
...,...,...,...,...,...
289798,656829039a79476d415dc08b,Финансы,СПБ Биржа решила получить лицензию OFAC для ра...,Основной стратегией биржи по разблокировке акт...,СПБ Биржа рассчитывает получить лицензию Управ...
289799,656829039a79476d415dc08b,Финансы,СПБ Биржа решила получить лицензию OFAC для ра...,Основной стратегией биржи по разблокировке акт...,СПБ Биржа рассчитывает получить лицензию Управ...
289800,656829039a79476d415dc08b,Финансы,СПБ Биржа решила получить лицензию OFAC для ра...,Основной стратегией биржи по разблокировке акт...,СПБ Биржа рассчитывает получить лицензию Управ...
289801,656829039a79476d415dc08b,Финансы,СПБ Биржа решила получить лицензию OFAC для ра...,Основной стратегией биржи по разблокировке акт...,СПБ Биржа рассчитывает получить лицензию Управ...


In [10]:
rbk_df['title'].iloc[0], rbk_df['body'].iloc[0]

('Токаев ввел чрезвычайное положение в Мангистауской области и Алма-Ате',
 'Чрезвычайное положение вступит в силу 5 января и продлится две недели. В стране 2 января вспыхнули протесты после двукратного повышения цен на автомобильный газ')

In [11]:
rbk_df['text'].iloc[0]

'Президент Казахстана Касым-Жомарт Токаев подписал указы о введении чрезвычайного положения в Мангистауской области и Алма-Ате, пишет Tengrinews. Чрезвычайное положение будет действовать с 01:30 5 января по местному времени до 19 января. На западе Казахстана из-за роста цен на автомобильный газ начались протесты, которые вскоре охватили другие регионы страны уже после обещания снизить цены. 1 января газ подорожал с 60 тенге (10,2 руб.) до 120 (20,4 руб.). После двух дней митингов владельцы автозаправок в Мангистауской области снизили цену до 80–90 тенге, но митингующие посчитали это недостаточным. Протесты также начались в Нур-Султане, Караганде, Атырау, Уральске, Талдыкоргане, Кызылорде и Шымкенте.  На площади Республики в Алма-Ате\xa0силовики начали применять светошумовые гранаты против протестующих. Sputnik передает, что собравшиеся начали громить полицейские машины и избили сотрудника полиции. Большинство демонстрантов удалось оттеснить с площади. В городе перестали работать мобиль

In [12]:
rbk_df['overview'].iloc[0]

'Чрезвычайное положение вступит в силу 5 января и продлится две недели. В стране 2 января вспыхнули протесты после двукратного повышения цен на автомобильный газ'

In [13]:
rbk_df[rbk_df['overview'] != rbk_df['body']]

,id,project,project_nick,type,category,title,body,publish_date,publish_date_t,fronturl,...,PER,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,date_y,sentiment
11,61d4add39a79478538331218,РБК,rbcnews,short_news,Политика,Ермак позвонил советникам глав ФРГ и Франции п...,NaN,2022-01-04T23:38:37+03:00,1641328717,https://www.rbc.ru/rbcfreenews/61d4add39a79478...,...,60,20220105,00:00,"1,305.5","1,306.2","1,304.5","1,305.9","8,080",2022-01-05 00:00:00,отрицательный
12,61d4add39a79478538331218,РБК,rbcnews,short_news,Политика,Ермак позвонил советникам глав ФРГ и Франции п...,NaN,2022-01-04T23:38:37+03:00,1641328717,https://www.rbc.ru/rbcfreenews/61d4add39a79478...,...,60,20220105,00:00,"1,305.5","1,306.2","1,304.5","1,305.9","8,080",2022-01-05 00:00:00,отрицательный
13,61d4add39a79478538331218,РБК,rbcnews,short_news,Политика,Ермак позвонил советникам глав ФРГ и Франции п...,NaN,2022-01-04T23:38:37+03:00,1641328717,https://www.rbc.ru/rbcfreenews/61d4add39a79478...,...,60,20220105,00:00,"1,305.5","1,306.2","1,304.5","1,305.9","8,080",2022-01-05 00:00:00,отрицательный
14,61d4add39a79478538331218,РБК,rbcnews,short_news,Политика,Ермак позвонил советникам глав ФРГ и Франции п...,NaN,2022-01-04T23:38:37+03:00,1641328717,https://www.rbc.ru/rbcfreenews/61d4add39a79478...,...,60,20220105,00:00,"1,305.5","1,306.2","1,304.5","1,305.9","8,080",2022-01-05 00:00:00,отрицательный
15,61d4add39a79478538331218,РБК,rbcnews,short_news,Политика,Ермак позвонил советникам глав ФРГ и Франции п...,NaN,2022-01-04T23:38:37+03:00,1641328717,https://www.rbc.ru/rbcfreenews/61d4add39a79478...,...,60,20220105,00:00,"1,305.5","1,306.2","1,304.5","1,305.9","8,080",2022-01-05 00:00:00,отрицательный
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289748,656835439a7947042216d96b,РБК,rbcnews,short_news,Политика,"В Совфеде после смерти Киссинджера заявили, чт...",NaN,2023-11-30T10:53:38+03:00,1701330818,https://www.rbc.ru/rbcfreenews/656835439a79470...,...,60,20231130,11:00,480.1,487.2,463.3,463.7,"641,500",2023-11-30 11:00:00,отрицательный
289749,656835439a7947042216d96b,РБК,rbcnews,short_news,Политика,"В Совфеде после смерти Киссинджера заявили, чт...",NaN,2023-11-30T10:53:38+03:00,1701330818,https://www.rbc.ru/rbcfreenews/656835439a79470...,...,60,20231130,11:00,480.1,487.2,463.3,463.7,"641,500",2023-11-30 11:00:00,отрицательный
289750,656835439a7947042216d96b,РБК,rbcnews,short_news,Политика,"В Совфеде после смерти Киссинджера заявили, чт...",NaN,2023-11-30T10:53:38+03:00,1701330818,https://www.rbc.ru/rbcfreenews/656835439a79470...,...,60,20231130,11:00,480.1,487.2,463.3,463.7,"641,500",2023-11-30 11:00:00,отрицательный
289751,656835439a7947042216d96b,РБК,rbcnews,short_news,Политика,"В Совфеде после смерти Киссинджера заявили, чт...",NaN,2023-11-30T10:53:38+03:00,1701330818,https://www.rbc.ru/rbcfreenews/656835439a79470...,...,60,20231130,11:00,480.1,487.2,463.3,463.7,"641,500",2023-11-30 11:00:00,отрицательный


In [14]:
rbk_df.shape

(289803, 30)

In [15]:
lenta_df[['docid', 'title', 'text', 'rightcol', 'snippet']]

,docid,title,text,rightcol,snippet
0,1181788,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 Евгения Н...,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 ... поток...
1,1181788,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 Евгения Н...,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 ... поток...
2,1181788,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 Евгения Н...,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 ... поток...
3,1181788,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 Евгения Н...,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 ... поток...
4,1181788,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 Евгения Н...,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 ... поток...
...,...,...,...,...,...
114703,1535261,Путин разрешил «Лукойлу» выкупить долю UROC Li...,Фото: Sputnik / Reuters Александра Синицына Пр...,Путин разрешил «Лукойлу» выкупить долю UROC Li...,Фото: Sputnik / Reuters Александра ... энергет...
114704,1535261,Путин разрешил «Лукойлу» выкупить долю UROC Li...,Фото: Sputnik / Reuters Александра Синицына Пр...,Путин разрешил «Лукойлу» выкупить долю UROC Li...,Фото: Sputnik / Reuters Александра ... энергет...
114705,1535261,Путин разрешил «Лукойлу» выкупить долю UROC Li...,Фото: Sputnik / Reuters Александра Синицына Пр...,Путин разрешил «Лукойлу» выкупить долю UROC Li...,Фото: Sputnik / Reuters Александра ... энергет...
114706,1535261,Путин разрешил «Лукойлу» выкупить долю UROC Li...,Фото: Sputnik / Reuters Александра Синицына Пр...,Путин разрешил «Лукойлу» выкупить долю UROC Li...,Фото: Sputnik / Reuters Александра ... энергет...


In [16]:
lenta_df['text'].iloc[0]

'Фото: Nikolai Ryutin / Nord Stream 2 Евгения Наумова Заместитель главы МИД России Александр Панкин заявил, что процесс сертификации газопровода «Северный поток-2» может занять длительное время. Примерные сроки завершения этой процедуры он назвал в разговоре с РИА Новости. Панкин отметил, что регламенты процедуры позволяют растянуть сертификацию «Северного потока-2» на много месяцев. По его мнению, этот процесс может продлиться до лета 2022 года. Дипломат не исключил, что процедура может быть осложнена со стороны властей США. «Почему бы не представить, что американцы скажут немцам ни грамма российского газа по "Северному потоку" не покупать? Он может быть сертифицирован, а могут и не просертифицировать. Могут что угодно применить», — сказал Панкин и добавил, что козни от Вашингтона «могут быть на каждом шагу». 2 января сообщалось о том, что в Германии допустили введение санкций против «Северного потока-2» в случае, если Россия вторгнется на Украину. Уточнялось, что с предложением рассм

In [17]:
lenta_df['rightcol'].iloc[0]

'В МИД России назвали сроки сертификации «Северного потока-2»'

In [18]:
lenta_df.shape

(114708, 29)

In [19]:
lenta_df['snippet'].iloc[90]

'Фото: Mike Segar / Reuters Марина Совина ... превышает три триллиона долларов. Об этом свидетельствуют данные торгов на Нью-Йоркской фондовой бирже. Акции компании дорожают почти на три процента.'

In [18]:
lenta_df.columns

Index(['docid', 'url', 'title', 'modified', 'lastmodtime', 'type', 'domain',
       'status', 'part', 'bloc', 'tags', 'image_url', 'pubdate', 'text',
       'rightcol', 'snippet', 'date_x', 'key_date', 'TICKER', 'PER', 'DATE',
       'TIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL', 'date_y', 'sentiment'],
      dtype='object')

In [ ]:
lenta_df.rightcol.iloc[0]

'В МИД России назвали сроки сертификации «Северного потока-2»'

In [ ]:
lenta_df.text.iloc[0]

'Фото: Nikolai Ryutin / Nord Stream 2 Евгения Наумова Заместитель главы МИД России Александр Панкин заявил, что процесс сертификации газопровода «Северный поток-2» может занять длительное время. Примерные сроки завершения этой процедуры он назвал в разговоре с РИА Новости. Панкин отметил, что регламенты процедуры позволяют растянуть сертификацию «Северного потока-2» на много месяцев. По его мнению, этот процесс может продлиться до лета 2022 года. Дипломат не исключил, что процедура может быть осложнена со стороны властей США. «Почему бы не представить, что американцы скажут немцам ни грамма российского газа по "Северному потоку" не покупать? Он может быть сертифицирован, а могут и не просертифицировать. Могут что угодно применить», — сказал Панкин и добавил, что козни от Вашингтона «могут быть на каждом шагу». 2 января сообщалось о том, что в Германии допустили введение санкций против «Северного потока-2» в случае, если Россия вторгнется на Украину. Уточнялось, что с предложением рассм

In [ ]:
#lenta_df.drop_duplicates(subset=['docid'], inplace=True, ignore_index=True)
lenta_df = lenta_df.dropna(subset=['text']).reset_index(drop=True)
lenta_df.shape

(114511, 29)

In [ ]:
lenta_df[lenta_df.columns[:20]].head()

,Unnamed: 0,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet,date_x,key_date,TICKER
0,0,1181788,https://lenta.ru/news/2022/01/03/sroki/,В МИД России назвали сроки сертификации «Север...,1641201720,1641204328,1,1,0,0,4,[8],https://icdn.lenta.ru/images/2022/01/03/12/202...,1641201720,Фото: Nikolai Ryutin / Nord Stream 2 Евгения Н...,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 ... поток...,2022-01-03 09:22:00,2022-01-03 06 | 2022-01-03 09,SBER
1,1,1181788,https://lenta.ru/news/2022/01/03/sroki/,В МИД России назвали сроки сертификации «Север...,1641201720,1641204328,1,1,0,0,4,[8],https://icdn.lenta.ru/images/2022/01/03/12/202...,1641201720,Фото: Nikolai Ryutin / Nord Stream 2 Евгения Н...,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 ... поток...,2022-01-03 09:22:00,2022-01-03 06 | 2022-01-03 09,ROSN
2,2,1181788,https://lenta.ru/news/2022/01/03/sroki/,В МИД России назвали сроки сертификации «Север...,1641201720,1641204328,1,1,0,0,4,[8],https://icdn.lenta.ru/images/2022/01/03/12/202...,1641201720,Фото: Nikolai Ryutin / Nord Stream 2 Евгения Н...,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 ... поток...,2022-01-03 09:22:00,2022-01-03 06 | 2022-01-03 09,MOEX
3,3,1181788,https://lenta.ru/news/2022/01/03/sroki/,В МИД России назвали сроки сертификации «Север...,1641201720,1641204328,1,1,0,0,4,[8],https://icdn.lenta.ru/images/2022/01/03/12/202...,1641201720,Фото: Nikolai Ryutin / Nord Stream 2 Евгения Н...,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 ... поток...,2022-01-03 09:22:00,2022-01-03 06 | 2022-01-03 09,YNDX
4,4,1181788,https://lenta.ru/news/2022/01/03/sroki/,В МИД России назвали сроки сертификации «Север...,1641201720,1641204328,1,1,0,0,4,[8],https://icdn.lenta.ru/images/2022/01/03/12/202...,1641201720,Фото: Nikolai Ryutin / Nord Stream 2 Евгения Н...,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 ... поток...,2022-01-03 09:22:00,2022-01-03 06 | 2022-01-03 09,PIKK


In [ ]:
from collections import Counter

def drop_duplicates(txt):
    try:
        txt = re.split('\. |: |\n', txt)
        counter_txt = Counter(txt)
        if len(txt) < 2: return ''
        i = len(txt)-2
        while counter_txt[txt[i]] > 1:
            i-=1
    except:
        return ''
    if i==len(txt)-2:
        return " ".join(txt)
    return " ".join(txt[:i+1])

def replace_numbers(txt):
    phone_pattern = r'\b(\+)?([78]\d{10})\b'
    date_pattern = r'\d{1,2}\.\d{1,2}\.\d{2,4}'
    time_pattern = r'(\d{1,2}\:)?\d{1,2}\:\d{1,2}'
    passport_pattern = r'\d{4} \d{6}'
    nums_pattern = r'(\d{1,2} )?\d+(\,\d{1,2})?'
    trash_pattern = r'УКАЖИ'

    new_text = re.sub(phone_pattern, ' phone_number ', txt)
    new_text = re.sub(date_pattern, 'date ', new_text)
    new_text = re.sub(time_pattern, 'time ', new_text)
    new_text = re.sub(passport_pattern, 'passport ', new_text)
    new_text = re.sub(nums_pattern, 'number ', new_text)
    new_text = re.sub(trash_pattern, '', new_text)

    new_text = re.sub(r'[^\w\s]','', new_text).lower()
    new_text = re.sub(r'[\\]',' ', new_text).lower()
    return new_text

def full_describe_request(txt, add_feature_txt, add_txt):
    return txt + ' [' + str(add_feature_txt) + ' ]:' + str(add_txt)

def parse(series, features, text_col = 'text'):
    txt = '' if not isinstance(series[text_col], str) else series[text_col]
    uniq_txt = drop_duplicates(txt)

    for feature in features:
        uniq_txt = full_describe_request(uniq_txt, feature, series[feature])
    txt = replace_numbers(uniq_txt)
    return txt.strip("\n")

In [ ]:
lenta_df['Описание заявки'] = lenta_df.progress_apply(
    parse,
    args = (
     ['title'],
    ),
    axis = 1,
)

request_txts = lenta_df['Описание заявки']

lenta_df['Описание заявки'].iloc[0]

  0%|          | 0/114511 [00:00<?, ?it/s]

'фото nikolai ryutin  nord stream number  евгения наумова заместитель главы мид россии александр панкин заявил что процесс сертификации газопровода северный потокnumber  может занять длительное время примерные сроки завершения этой процедуры он назвал в разговоре с риа новости панкин отметил что регламенты процедуры позволяют растянуть сертификацию северного потокаnumber  на много месяцев по его мнению этот процесс может продлиться до лета number  года дипломат не исключил что процедура может быть осложнена со стороны властей сша почему бы не представить что американцы скажут немцам ни грамма российского газа по северному потоку не покупать он может быть сертифицирован а могут и не просертифицировать могут что угодно применить  сказал панкин и добавил что козни от вашингтона могут быть на каждом шагу number  января сообщалось о том что в германии допустили введение санкций против северного потокаnumber  в случае если россия вторгнется на украину уточнялось что с предложением рассмотрет

**Target**

In [ ]:
sentences= df['Описание заявки'].values
labels = df['TARGET_NUM'].values

## Data preparing ##

In [ ]:
import os

tokenizer = AutoTokenizer.from_pretrained(
    "ai-forever/sbert_large_nlu_ru",
    do_lower_case=True,
)

replace_tokens = {
        '[unused1]': 'phone_number',
        '[unused2]': 'date',
        '[unused3]': 'passport',
        '[unused4]': 'number',
        '[unused5]': 'premium',
}

vocab = tokenizer.get_vocab()

for old_token, new_token in replace_tokens.items():
    idx = vocab[old_token]
    del vocab[old_token]
    vocab[new_token] = idx

# Тут надо отсортировать, иначе далее при инициализации токенайзера порядок ломается
vocab = pd.Series(vocab).sort_values()
vocab = vocab.index.tolist()

with open('vocab.tmp', 'w', encoding = 'utf-8') as tmp_vocab_file:
    tmp_vocab_file.write('\n'.join(vocab))

tokenizer = AutoTokenizer.from_pretrained(
    "ai-forever/sbert_large_nlu_ru",
    vocab_file = 'vocab.tmp', do_lower_case=True, do_basic_tokenize=False
)

os.remove('vocab.tmp')

In [ ]:
print('Original: ', sentences[0])
print('Tokenised: ', tokenizer.tokenize(sentences[0], add_special_tokens=True))
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0], add_special_tokens=True)))

In [ ]:
max_len = 0
input_ids_all = []

for sent in tqdm(sentences):
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
    input_ids_all.append(input_ids)

max_len

In [ ]:
input_ids = []
attention_masks = []

for sent in tqdm(sentences):
  encoded_dict = tokenizer.encode_plus(
      sent,
      add_special_tokens=True,
      max_length=256,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
  )
  input_ids.append(encoded_dict['input_ids'])
  attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

print(sentences[0])
print(input_ids[0])

In [ ]:
len(input_ids), len(attention_masks), len(labels)

In [ ]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)
train_size = int(0.8 * len(dataset))
val_size = int(0.3 * (len(dataset) - train_size))
test_size = len(dataset) - train_size - val_size

train_data, val_data = random_split(dataset, [train_size, val_size + test_size])
val_data, test_data = random_split(val_data, [val_size, test_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} testing samples'.format(test_size))

In [ ]:
batch_size = 32

train_dataloader = DataLoader(
    train_data,
    sampler = RandomSampler(train_data),
    batch_size=batch_size
)

validation_dataloader = DataLoader(
    val_data,
    sampler = SequentialSampler(val_data),
    batch_size=batch_size
)

**BERT prepare**

In [ ]:
import torch

t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved

a, b = torch.cuda.mem_get_info()
print(a / 1024 / 1024 / 1024)
print(b / 1024 / 1024 / 1024)

In [ ]:
#model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

model = BertForSequenceClassification.from_pretrained(
    "ai-forever/sbert_large_nlu_ru", #'bert-base-uncased',
    num_labels = 16,
    output_attentions = False,
    output_hidden_states = False,
)

model.cuda()

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                  )

In [ ]:
epochs = 3

total_steps = len(train_dataloader) * epochs
scheduler =  get_linear_schedule_with_warmup(optimizer,
                                             num_warmup_steps = 0,
                                             num_training_steps = total_steps)

## Training loop ##

**Training**:
*   Распаковать обучающие данные и лейблы
*   Загрузить данные на GPU для ускорения
*   Занулить градиенты с предыдущего шага
*   Forward pass (скормить данные в нейросеть и пробросить их вперед)
*   Backward pass (back propagation - посчитать градиенты по всем параметрам с помощью обратного распространения ошибки)
*   Обновить параметры с помощью optimizer.step()
*   Посчитать статистики, чтобы следить за обучением ⛅

**Evaluation**:

*   Распаковать валидационные данные и лейблы
*   Загрузить данные на GPU для ускорения
*   Forward pass (скормить данные в нейросеть и пробросить вперед)
*   Посчиатть loss и статистики для валиданционных данных, чтобы следить за обучением

In [ ]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved

a, b = torch.cuda.mem_get_info()
print(a / 1024 / 1024 / 1024)
print(b / 1024 / 1024 / 1024)

In [ ]:
def flat_accuracy(preds, labels):
  pred_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return np.sum(pred_flat == labels_flat) / len(labels_flat)

def  format_time(elapsed):
  elapsed_rounded = int(round(elapsed))
  return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []

total_t0 = time.time()

for epoch_i in range(0, epochs):
  print('')
  print('======= EPOCH {:} / {:} ======'.format(epoch_i + 1, epochs))
  print('Training...')

  t0 = time.time()
  total_train_loss = 0
  model.train()

  for step, batch in enumerate(train_dataloader):
    if step % 40 == 0 and not step == 0:
      elapsed = format_time(time.time() - t0)
      #Report progress
      print('   Batch {0:>5} of {1:>5}. Elapsed: {2:} Loss: {3:.4f}  Average_Train_Loss: {4:.4f}'.format(step, len(train_data), elapsed, res['loss'], total_train_loss/(batch_size * (step + 1))))

    #batch  contains 3 pytorch tensors: [0] input ids, [1] aattention masks, [2] labels
    #print(batch[0])
    #print(batch[1])
    #print(batch[2])
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    model.zero_grad()
    #Forward pass
    res = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask,labels=b_labels)
    loss = res['loss']
    logits = res['logits'] #вероятности классов для батча

    total_train_loss += loss.item()
    loss.backward()
    #нормы градиентов обрезаем до 1.0, чтобы предотвратить проблему взрывающихся градиентов
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    optimizer.step() #обновление весов
    scheduler.step()

    avg_train_loss = total_train_loss/len(train_dataloader)
    train_time = format_time(time.time() - t0)

  print(' Average training loss: {0:.2f}'.format(avg_train_loss))
  print(' Training epoch took: {:}'.format(train_time))


  #Validation
  print("Running Validation...")

  t0 = time.time()
  model.eval()
  total_eval_accuracy = 0
  total_eval_loss = 0
  nb_eval_steps = 0

  for batch in validation_dataloader:
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)

      with torch.no_grad():
          # Forward pass, calculate logit predictions.
          result = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels,
                        return_dict=True)
      loss = result.loss
      logits = result.logits
      total_eval_loss += loss.item()
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
      total_eval_accuracy += flat_accuracy(logits, label_ids)

  avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
  print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

  avg_val_loss = total_eval_loss / len(validation_dataloader)
  validation_time = format_time(time.time() - t0)

  print("  Validation Loss: {0:.2f}".format(avg_val_loss))
  #print("  Validation Loss: {0:.2f}".format(avg_val_loss))
  print("  Validation took: {:}".format(validation_time))

  training_stats.append(
      {
          'epoch': epoch_i + 1,
          'Training Loss': avg_train_loss,
          'Valid. Loss': avg_val_loss,
          'Valid. Accur.': avg_val_accuracy,
          'Training Time': train_time,
          'Validation Time': validation_time
      }
  )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))